# CSCB Project

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 
import scanpy as sc
import scipy as sp
import numpy as np
import anndata
import pySingleCellNet as pySCN
sns.set_style('white')

/Users/wvcui/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/wvcui/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


## Read in Data

In [2]:
# read in data
#mcm raw is our engineered mouse cardiomyocytes
adata_mcm_raw=sc.read_h5ad('/Users/wvcui/Desktop/OneDrive - Johns Hopkins/Sp22/Comp SC/final proj/direct_conversion_subsampled_5k_raw.h5ad')

#this is the reference dataset from the tabula muris study
adata_tabmuris=sc.read_h5ad('/Users/wvcui/Desktop/OneDrive - Johns Hopkins/Sp22/Comp SC/final proj/TM_FACS_heart.h5ad')

In [3]:
# metadata was labeled as batch, translating this to timepoint
adata_mcm_raw.obs['Timepoint (days)']=adata_mcm_raw.obs['batch'].map({'1':'-1','2':'1','3':'2','4':'3','5':'7','6':'14'})

In [8]:
adata_tabmuris.var

""
0610005C13Rik
0610007C21Rik
0610007L01Rik
0610007N19Rik
0610007P08Rik
...
Zzef1
Zzz3
a
l7Rn6


In [6]:
human = sc.read_h5ad('/Users/wvcui/Desktop/OneDrive - Johns Hopkins/Sp22/Comp SC/final proj/human_engineered_cardiomyocytes/hIPSC_CM_orlova_GSM4443382.h5ad')

In [7]:
human.var

,gene_ids
RP11-34P13.3,ENSG00000243485
FAM138A,ENSG00000237613
OR4F5,ENSG00000186092
RP11-34P13.7,ENSG00000238009
RP11-34P13.8,ENSG00000239945
...,...
AC233755.2,ENSG00000277856
AC233755.1,ENSG00000275063
AC240274.1,ENSG00000271254
AC213203.1,ENSG00000277475


In [7]:
#mt and ribo genes, qc metrics
adata_mcm_raw.var['mt']= adata_mcm_raw.var_names.str.startswith(("mt-"))
adata_mcm_raw.var['ribo'] = adata_mcm_raw.var_names.str.startswith(("Rps","Rpl"))
sc.pp.calculate_qc_metrics(adata_mcm_raw, qc_vars=['ribo', 'mt'], percent_top=None, log1p=False, inplace=True)

In [8]:
#mt and ribo genes, qc metrics
human.var['mt']= human.var_names.str.startswith(("mt-"))
human.var['ribo'] = human.var_names.str.startswith(("Rps","Rpl"))
sc.pp.calculate_qc_metrics(human, qc_vars=['ribo', 'mt'], percent_top=None, log1p=False, inplace=True)

## Q1

In [10]:
#finding matching genes between datasets
genesTrain = adata_tabmuris.var_names
genesQuery = human.var_names
cgenes = genesTrain.intersection(genesQuery)
#around 14k genes remanining

In [11]:
#subsetting for common genes
adata_reference_cgene = adata_tabmuris[:,cgenes]
human = human[:,cgenes].copy()

#following the pyscCellNet vignette on github. im guessing cells with more than 500 
#adata_mcm_classifier = adata_mcm_raw[adata_mcm_raw.obs["n_genes_by_counts"]>=500,:].copy()

human_classifier = human[human.obs["n_genes_by_counts"]>=500,:].copy()



In [12]:
adata_reference_cgene.obs['cell_ontology_class'].unique()

['endothelial cell', 'leukocyte', 'fibroblast', 'myofibroblast cell', 'endocardial cell', 'cardiac muscle cell', 'smooth muscle cell']
Categories (7, object): ['cardiac muscle cell', 'endocardial cell', 'endothelial cell', 'fibroblast', 'leukocyte', 'myofibroblast cell', 'smooth muscle cell']

In [17]:
expTrain, expVal = pySCN.splitCommonAnnData(adata_reference_cgene, ncells=200,dLevel='cell_ontology_class')



fibroblast : 
2119
cardiac muscle cell : 
133
myofibroblast cell : 
178
endothelial cell : 
1177
endocardial cell : 
165
leukocyte : 
523
smooth muscle cell : 
42


In [18]:
[cgenesA, xpairs, tspRF] = pySCN.scn_train(expTrain, nTopGenes = 100, nRand = 0,
                                           nTrees = 1000 ,nTopGenePairs = 100, dLevel = 'cell_ontology_class', 
                                           stratify=True, limitToHVG=True)

HVG


KeyError: "['A1.B000633.3_56_F.1.1', 'A1.B000634.3_56_F.1.1', 'A1.B002427.3_39_F.1.1', 'A1.B002428.3_38_F.1.1', 'A10.MAA000398.3_9_M.1.1', 'A10.MAA000586.3_8_M.1.1', 'A10.MAA000936.3_9_M.1.1', 'A11.B000636.3_56_F.1.1', 'A11.B002010.3_39_F.1.1', 'A11.MAA000398.3_9_M.1.1', 'A12.B000636.3_56_F.1.1', 'A12.MAA100037.3_11_M.1.1', 'A13.MAA000398.3_9_M.1.1', 'A13.MAA000903.3_11_M.1.1', 'A14.B000633.3_56_F.1.1', 'A14.B002010.3_39_F.1.1', 'A16.B000633.3_56_F.1.1', 'A16.B002423.3_39_F.1.1', 'A16.B002428.3_38_F.1.1', 'A16.MAA000936.3_9_M.1.1', 'A17.B000636.3_56_F.1.1', 'A18.B002011.3_38_F.1.1', 'A19.B000633.3_56_F.1.1', 'A19.B002428.3_38_F.1.1', 'A19.B002429.3_38_F.1.1', 'A19.MAA000903.3_11_M.1.1', 'A2.MAA000587.3_8_M.1.1', 'A2.MAA000920.3_11_M.1.1', 'A2.MAA100037.3_10_M.1.1', 'A20.B000412.3_56_F.1.1', 'A20.MAA000586.3_8_M.1.1', 'A21.B002429.3_38_F.1.1', 'A22.MAA000398.3_9_M.1.1', 'A4.MAA000918.3_10_M.1.1', 'A6.MAA000918.3_10_M.1.1', 'A7.B000633.3_56_F.1.1', 'A7.B002423.3_39_F.1.1', 'A7.MAA000400.

In [15]:
#classifier on held out test set
plt.figure(dpi=1000)
adVal = pySCN.scn_classify(expVal, cgenesA, xpairs, tspRF, nrand = 0)
assessment =  pySCN.assess_comm(expTrain, adVal, resolution = 0.005, nRand = 0, dLevelSID = "cell", classTrain = "cell_ontology_class", classQuery = "cell_ontology_class")

pySCN.plot_PRs(assessment)
plt.show()
plt.savefig("pyscn_testsetprc.png") #save as png

NameError: name 'cgenesA' is not defined

<Figure size 6000x4000 with 0 Axes>

In [ ]:
#use classifier on old hsc data
adata_mcm_classified = pySCN.scn_classify(adata_mcm_classifier, cgenesA, xpairs, tspRF, nrand = 0)



In [ ]:
adata_mcm_classified.obs

In [ ]:
#use classifier on human data

human_classified = pySCN.scn_classify(human, cgenesA, xpairs, tspRF, nrand = 0)
human_classified.obs

In [ ]:
ax = sc.pl.heatmap(human_classified, human_classified.var_names.values, groupby='SCN_class', cmap='viridis', dendrogram=False, swap_axes=True)


In [ ]:
human_clusters=pd.read_csv('/Users/wvcui/Desktop/OneDrive - Johns Hopkins/Sp22/Comp SC/final proj/clusters.tsv',sep='\t')

In [ ]:
human_clusters

In [ ]:
human_class = human_classified.copy()
sc.pp.filter_genes(human_class, min_cells=5)
sc.pp.normalize_per_cell(human_class, counts_per_cell_after=1e4)
sc.pp.log1p(human_class)

sc.pp.highly_variable_genes(human_class, min_mean=0.0125, max_mean=4, min_disp=0.5)

human_class.raw = human_class
sc.pp.scale(human_class, max_value=10)
sc.tl.pca(human_class, n_comps=100)

sc.pl.pca_variance_ratio(human_class, 100)

In [ ]:
npcs = 20
sc.pp.neighbors(human_class, n_neighbors=10, n_pcs=npcs)
sc.tl.leiden(human_class,.1)
sc.tl.umap(human_class, .5)
sc.pl.umap(human_class, color=["leiden", "SCN_class"], alpha=.9, s=15, legend_loc='on data')

In [ ]:
props={}
for i in adata_mcm_classified.obs['Timepoint (days)'].unique():
    props[i]={}
    for j in adata_mcm_classified[adata_mcm_classified.obs['Timepoint (days)']==i].obs['SCN_class'].unique():
        props[i][j]=sum(adata_mcm_classified[adata_mcm_classified.obs['Timepoint (days)']==i].obs['SCN_class']==j)

In [ ]:
scn_calls=pd.DataFrame(props).fillna(0)
scn_calls.update(scn_calls.div(scn_calls.sum(axis=0),axis=1))
scn_calls=scn_calls[['-1','1','2','3','7','14']]

In [ ]:
scn_calls

In [ ]:
plt.figure(figsize=(10, 6), dpi=80)

plt.bar(scn_calls.columns,scn_calls.loc['cardiac muscle cell'],color='red')
#plt.bar(scn_calls.columns, scn_calls.loc['rand'], bottom=scn_calls.loc['cardiac muscle cell'], color='grey')
plt.bar(scn_calls.columns, scn_calls.loc['leukocyte'], bottom=scn_calls.loc['cardiac muscle cell'], color='pink')
plt.bar(scn_calls.columns, scn_calls.loc['fibroblast'], bottom=scn_calls.loc['leukocyte']+scn_calls.loc['cardiac muscle cell'], color='green')
plt.bar(scn_calls.columns, scn_calls.loc['endothelial cell'], bottom=scn_calls.loc['leukocyte']+scn_calls.loc['cardiac muscle cell']+scn_calls.loc['fibroblast'], color='maroon')
plt.bar(scn_calls.columns, scn_calls.loc['endocardial cell'], bottom=scn_calls.loc['leukocyte']+scn_calls.loc['cardiac muscle cell']+scn_calls.loc['fibroblast']+scn_calls.loc['endothelial cell'], color='purple')


plt.legend(["cardiac muscle cell", "leukocyte", "fibroblast",'endothelial cell','endocardial cell'],loc='upper right',bbox_to_anchor=(1.25, 1.0))
plt.xlabel('Days in Vitro (DIV)')
plt.ylabel('Cell Type Proportion')
plt.title('pySCN cell type calls of Mouse Engineered Cardiomyocyte transdifferentiation Experiment')
plt.savefig("pyscn_stackedbarplot.png") #save as png


In [ ]:
props={}
for i in adata_mcm_classified.obs['Timepoint (days)'].unique():
    props[i]={}
    for j in adata_mcm_classified[adata_mcm_classified.obs['Timepoint (days)']==i].obs['SCN_class'].unique():
        props[i][j]=sum(adata_mcm_classified[adata_mcm_classified.obs['Timepoint (days)']==i].obs['SCN_class']==j)

In [ ]:
scn_calls=pd.DataFrame(props).fillna(0)
scn_calls.update(scn_calls.div(scn_calls.sum(axis=0),axis=1))
scn_calls=scn_calls[['-1','1','2','3','7','14']]

In [ ]:
scn_calls

In [ ]:
plt.figure(figsize=(10, 6), dpi=80)

plt.bar(scn_calls.columns,scn_calls.loc['cardiac muscle cell'],color='red')
#plt.bar(scn_calls.columns, scn_calls.loc['rand'], bottom=scn_calls.loc['cardiac muscle cell'], color='grey')
plt.bar(scn_calls.columns, scn_calls.loc['leukocyte'], bottom=scn_calls.loc['cardiac muscle cell'], color='pink')
plt.bar(scn_calls.columns, scn_calls.loc['fibroblast'], bottom=scn_calls.loc['leukocyte']+scn_calls.loc['cardiac muscle cell'], color='green')
plt.bar(scn_calls.columns, scn_calls.loc['endothelial cell'], bottom=scn_calls.loc['leukocyte']+scn_calls.loc['cardiac muscle cell']+scn_calls.loc['fibroblast'], color='maroon')
plt.bar(scn_calls.columns, scn_calls.loc['endocardial cell'], bottom=scn_calls.loc['leukocyte']+scn_calls.loc['cardiac muscle cell']+scn_calls.loc['fibroblast']+scn_calls.loc['endothelial cell'], color='purple')


plt.legend(["cardiac muscle cell", "leukocyte", "fibroblast",'endothelial cell','endocardial cell'],loc='upper right',bbox_to_anchor=(1.25, 1.0))
plt.xlabel('Days in Vitro (DIV)')
plt.ylabel('Cell Type Proportion')
plt.title('pySCN cell type calls of Mouse Engineered Cardiomyocyte transdifferentiation Experiment')
plt.savefig("pyscn_stackedbarplot.png") #save as png


## Q2

In [ ]:
adata_kannan=sc.read_h5ad('/Users/wvcui/Desktop/OneDrive - Johns Hopkins/Sp22/Comp SC/final proj/perinatal_reference_data/Kannan_perinatal_reference.h5ad')

In [ ]:
adata_kannan #raw counts

In [ ]:
adata_mcm_raw.obs['scn_class']=adata_mcm_classified.obs['SCN_class']
adata_mcm_classifier_mcmcalls=adata_mcm_raw[adata_mcm_raw.obs['scn_class']=='cardiac muscle cell']

In [ ]:
#finding matching genes between datasets
genesTrain = adata_kannan.var_names
genesQuery = adata_mcm_classifier_mcmcalls.var_names
cgenes = genesTrain.intersection(genesQuery)
#around 14k genes remanining

In [ ]:
#subsetting for common genes
adata_reference_cgene = adata_kannan[:,cgenes]
adata_mcm_classifier_mcmcalls = adata_mcm_classifier_mcmcalls[:,cgenes].copy()

#following the pyscCellNet vignette on github. im guessing cells with more than 500 
adata_mcm_classifier_mcmcalls = adata_mcm_classifier_mcmcalls[adata_mcm_classifier_mcmcalls.obs["n_genes_by_counts"]>=500,:].copy()





In [ ]:
adata_reference_cgene.obs['timepoint'].unique()

In [ ]:
expTrain, expVal = pySCN.splitCommonAnnData(adata_reference_cgene, ncells=200,dLevel='timepoint')




In [ ]:
[cgenesA, xpairs, tspRF] = pySCN.scn_train(expTrain, nTopGenes = 100, nRand = 0,
                                           nTrees = 1000 ,nTopGenePairs = 100, dLevel = 'timepoint', 
                                           stratify=True, limitToHVG=True)

In [ ]:
plt.figure(dpi=1000)
ax = sc.pl.heatmap(adVal, adVal.var_names.values, groupby='SCN_class', cmap='viridis', dendrogram=False, swap_axes=True)


In [ ]:
#use classifier on old hsc data
adata_mcm_classified = pySCN.scn_classify(adata_mcm_classifier_mcmcalls, cgenesA, xpairs, tspRF, nrand = 0)




In [ ]:
props={}
for i in adata_mcm_classified.obs['Timepoint (days)'].unique():
    props[i]={}
    for j in adata_mcm_classified[adata_mcm_classified.obs['Timepoint (days)']==i].obs['SCN_class'].unique():
        props[i][j]=sum(adata_mcm_classified[adata_mcm_classified.obs['Timepoint (days)']==i].obs['SCN_class']==j)

In [ ]:
scn_calls=pd.DataFrame(props).fillna(0)
scn_calls.update(scn_calls.div(scn_calls.sum(axis=0),axis=1))
#scn_calls=scn_calls[['-1','1','2','3','7','14']]

In [ ]:
scn_calls=scn_calls[['1','2','3','7','14']]

In [ ]:
plt.figure(figsize=(10, 6), dpi=80)

plt.bar(scn_calls.columns,scn_calls.loc['e14'],color='red')
#plt.bar(scn_calls.columns, scn_calls.loc['rand'], bottom=scn_calls.loc['cardiac muscle cell'], color='grey')
plt.bar(scn_calls.columns, scn_calls.loc['e18'], bottom=scn_calls.loc['e14'], color='pink')
plt.bar(scn_calls.columns, scn_calls.loc['p8'], bottom=scn_calls.loc['e14']+scn_calls.loc['e18'], color='green')
plt.bar(scn_calls.columns, scn_calls.loc['p11'], bottom=scn_calls.loc['e14']+scn_calls.loc['e18']+scn_calls.loc['p8'], color='maroon')
plt.bar(scn_calls.columns, scn_calls.loc['p14'], bottom=scn_calls.loc['e14']+scn_calls.loc['e18']+scn_calls.loc['p8']+scn_calls.loc['p11'], color='purple')
plt.bar(scn_calls.columns, scn_calls.loc['p18'], bottom=scn_calls.loc['e14']+scn_calls.loc['e18']+scn_calls.loc['p8']+scn_calls.loc['p11']+scn_calls.loc['p14'], color='grey')
plt.bar(scn_calls.columns, scn_calls.loc['p22'], bottom=scn_calls.loc['e14']+scn_calls.loc['e18']+scn_calls.loc['p8']+scn_calls.loc['p11']+scn_calls.loc['p14']+scn_calls.loc['p18'], color='black')
plt.bar(scn_calls.columns, scn_calls.loc['p35'], bottom=scn_calls.loc['e14']+scn_calls.loc['e18']+scn_calls.loc['p8']+scn_calls.loc['p11']+scn_calls.loc['p14']+scn_calls.loc['p18']+scn_calls.loc['p22'], color='blue')
plt.bar(scn_calls.columns, scn_calls.loc['p56'], bottom=scn_calls.loc['e14']+scn_calls.loc['e18']+scn_calls.loc['p8']+scn_calls.loc['p11']+scn_calls.loc['p14']+scn_calls.loc['p18']+scn_calls.loc['p22']+scn_calls.loc['p35'], color='yellow')





plt.legend(["e14", "e18", "p8",'p11','p14','p18','p22','p35','p56'],loc='upper right',bbox_to_anchor=(1.25, 1.0))
plt.xlabel('Days in Vitro (DIV)')
plt.ylabel('Cell Type Proportion')
plt.title('pySCN cell type calls of Mouse Engineered Cardiomyocyte transdifferentiation Experiment')
plt.savefig("pyscn_stackedbarplot.png") #save as png

